In [41]:
from tensorflow.keras.layers import Dense,InputLayer, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, models
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [42]:
# Cargar el dataset ya dividido en train/val/test usando el parámetro split de TFDS
(train, val, test), data_info = tfds.load("malaria",with_info=True,
                                        split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
                                        shuffle_files=True)

In [43]:
data_info

tfds.core.DatasetInfo(
    name='malaria',
    full_name='malaria/1.0.0',
    description="""
    The Malaria dataset contains a total of 27,558 cell images with equal instances
    of parasitized and uninfected cells from the thin blood smear slide images of
    segmented cells.
    """,
    homepage='https://lhncbc.nlm.nih.gov/publication/pub9932',
    data_dir='C:\\Users\\vpn\\tensorflow_datasets\\malaria\\1.0.0',
    file_format=tfrecord,
    download_size=337.08 MiB,
    dataset_size=317.62 MiB,
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    nondeterministic_order=False,
    splits={
        'train': <SplitInfo num_examples=27558, num_shards=4>,
    },
    citation="""@article{rajaraman2018pre,
      title={Pre-trained convolutional neural networks as feature extractors toward
      improved mal

In [44]:

def resize_only(example):
    image = example['image']
    label = example['label']
    # Redimensionar
    image = tf.image.resize(image, (224, 224))
    # Normalizar (opcional pero recomendado)
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

AUTOTUNE = tf.data.AUTOTUNE

train_dataset = (train
                .map(resize_only, num_parallel_calls=AUTOTUNE)
                .batch(32)
                .prefetch(AUTOTUNE))

val_dataset = (val
                .map(resize_only, num_parallel_calls=AUTOTUNE)
                .batch(32)
                .prefetch(AUTOTUNE))

test_dataset = (test
                .map(resize_only, num_parallel_calls=AUTOTUNE)
                .batch(32)
                .prefetch(AUTOTUNE))


In [45]:
# 🔹 Número de clases (Malaria: Parasitized / Uninfected → 2)
n_classes = 2

# 🔹 Definimos el modelo CNN
model = models.Sequential([
    # Entrada
    layers.Input(shape=(224, 224, 3)),
    #Bloque 1
    layers.Conv2D(10, (3,3), strides=1, padding='same',
                activation='relu', use_bias=True, name='conv1'),
    layers.Conv2D(10, (3,3), strides=1, padding='same',
                activation='relu', use_bias=True, name='conv2'),
    layers.MaxPooling2D((2,2), name='pool1'),#Reduce resolucion a la mitad
    # Bloque 2
    layers.Conv2D(10, (3,3), strides=1, padding='same',
                activation='relu', use_bias=True, name='conv3'),
    layers.Conv2D(10, (3,3), strides=1, padding='same',
                activation='relu', use_bias=True, name='conv4'),
    layers.MaxPooling2D((2,2), name='pool2'),#Reduce resolucion a la mitad
    #Reducción y capa densa
    layers.GlobalAveragePooling2D(name='gap'), #Promedia cada mapa en un vector de tamaño (batch_size, n_classes)
    layers.Dense(64, activation='relu', use_bias=True, name='fc1'),#Capa densa de 64 neyronas que procesa el vector de salida del gap
    layers.Dropout(0.3, name='dropout'),#Se apagan algunas neuronas el 30% en entrenamiento se encienden
    #Capa de salida (softmax porque es una clasificación)
    layers.Dense(n_classes, activation='softmax', use_bias=True, name='output')
])

#Compilacion del modelo
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            loss='sparse_categorical_crossentropy',   # porque las etiquetas son enteros 0/1
            metrics=['accuracy'])

#Resumen del modelo
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1 (Conv2D)              (None, 224, 224, 10)      280       
                                                                 
 conv2 (Conv2D)              (None, 224, 224, 10)      910       
                                                                 
 pool1 (MaxPooling2D)        (None, 112, 112, 10)      0         
                                                                 
 conv3 (Conv2D)              (None, 112, 112, 10)      910       
                                                                 
 conv4 (Conv2D)              (None, 112, 112, 10)      910       
                                                                 
 pool2 (MaxPooling2D)        (None, 56, 56, 10)        0         
                                                                 
______________________________________________________